In [1]:
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator # type: ignore
from tensorflow.keras.applications import MobileNetV2 # type: ignore
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D # type: ignore
from tensorflow.keras.models import Model # type: ignore
from tensorflow.keras.optimizers import Adam # type: ignore
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint # type: ignore
from config import img_width, img_height, num_classes

2024-05-31 03:50:10.960879: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# Paths
train_data_dir = "dataset/training"
test_data_dir = "dataset/test"
wip_models_dir = "models_wip"
models_dir = "models"
model_name = "chess_classifier_nightly"
current_best = os.path.join(wip_models_dir, f"{model_name}_best.keras")
final = os.path.join(models_dir, f"{model_name}.keras")

# Parameters
# img_width, img_height = 224, 224
batch_size = 32
epochs = 10
# num_classes = 13  # 6 pieces x 2 colors + 1 empty

In [3]:
os.makedirs(wip_models_dir, exist_ok=True)
os.makedirs(models_dir, exist_ok=True)

In [4]:
# Data normalization
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
)

test_datagen = ImageDataGenerator(rescale=1.0 / 255)

In [5]:
# Train and validation generators
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode="categorical",
)

validation_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode="categorical",
)

Found 200537 images belonging to 13 classes.
Found 50135 images belonging to 13 classes.


In [6]:
# Calculate steps_per_epoch and validation_steps
# steps_per_epoch = max(1, train_generator.samples // batch_size)
# validation_steps = max(1, validation_generator.samples // batch_size)
steps_per_epoch = max(1, len(train_generator) // batch_size)
validation_steps = max(1, len(validation_generator) // batch_size)

## Creating the model

Only using a single dense layer on top of MobileNet's, could probably improve the model by adding a few more.

In [7]:
# Load pre-trained MobileNetV2 model + higher level layers
base_model = MobileNetV2(
    weights="imagenet", include_top=False, input_shape=(img_width, img_height, 3)
)

# Add custom layers on top of the base model
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation="relu")(x)
predictions = Dense(num_classes, activation="softmax")(x)

# Final model
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze the base layers
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss="categorical_crossentropy",
    metrics=["accuracy"],
)

In [8]:
# Callbacks
early_stopping = EarlyStopping(
    monitor="val_loss", patience=10, restore_best_weights=True
)
model_checkpoint = ModelCheckpoint(
    current_best, monitor="val_loss", save_best_only=True
)

## Initial training

In [9]:
# Train the model
model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch,
    validation_data=validation_generator,
    validation_steps=validation_steps,
    epochs=epochs,
    callbacks=[early_stopping, model_checkpoint],
)

Epoch 1/10


/Users/ben/Projects/chessvision/.venv/lib/python3.11/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


195/195 ━━━━━━━━━━━━━━━━━━━━ 177s 884ms/step - accuracy: 0.8550 - loss: 0.4481 - val_accuracy: 0.9896 - val_loss: 0.0280
Epoch 2/10
195/195 ━━━━━━━━━━━━━━━━━━━━ 175s 900ms/step - accuracy: 0.9854 - loss: 0.0394 - val_accuracy: 0.9811 - val_loss: 0.0511
Epoch 3/10
195/195 ━━━━━━━━━━━━━━━━━━━━ 184s 945ms/step - accuracy: 0.9879 - loss: 0.0361 - val_accuracy: 0.9889 - val_loss: 0.0248
Epoch 4/10
195/195 ━━━━━━━━━━━━━━━━━━━━ 194s 994ms/step - accuracy: 0.9907 - loss: 0.0264 - val_accuracy: 0.9954 - val_loss: 0.0146
Epoch 5/10
195/195 ━━━━━━━━━━━━━━━━━━━━ 167s 857ms/step - accuracy: 0.9870 - loss: 0.0390 - val_accuracy: 0.9935 - val_loss: 0.0205
Epoch 6/10
195/195 ━━━━━━━━━━━━━━━━━━━━ 145s 745ms/step - accuracy: 0.9925 - loss: 0.0220 - val_accuracy: 0.9954 - val_loss: 0.0168
Epoch 7/10
195/195 ━━━━━━━━━━━━━━━━━━━━ 146s 752ms/step - accuracy: 0.9957 - loss: 0.0133 - val_accuracy: 0.9915 - val_loss: 0.0284
Epoch 8/10
195/195 ━━━━━━━━━━━━━━━━━━━━ 151s 774ms/step - accuracy: 0.9869 - loss: 0.04

## Fine tuning

Only the top layers were trainable in the initial training. In order to improve predictions, we unfreeze a few ImageNet layers and retrain the model with a much lower learning rate.

This might be counter-productive, not sure, needs more testing.

Either way, the best weights are used.

In [10]:
# Unfreeze some layers for fine-tuning
for layer in base_model.layers[-30:]:  # Unfreeze last 30 layers
    layer.trainable = True

# Recompile the model with a lower learning rate
model.compile(
    optimizer=Adam(learning_rate=1e-5),
    loss="categorical_crossentropy",
    metrics=["accuracy"],
)

# Fine-tune the model
model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch,
    validation_data=validation_generator,
    validation_steps=validation_steps,
    epochs=epochs,
    callbacks=[early_stopping, model_checkpoint],
)

Epoch 1/10
195/195 ━━━━━━━━━━━━━━━━━━━━ 199s 979ms/step - accuracy: 0.8641 - loss: 0.6391 - val_accuracy: 1.0000 - val_loss: 0.0025
Epoch 2/10
195/195 ━━━━━━━━━━━━━━━━━━━━ 186s 954ms/step - accuracy: 0.9798 - loss: 0.0619 - val_accuracy: 0.9967 - val_loss: 0.0105
Epoch 3/10
195/195 ━━━━━━━━━━━━━━━━━━━━ 188s 963ms/step - accuracy: 0.9891 - loss: 0.0386 - val_accuracy: 0.9993 - val_loss: 0.0033
Epoch 4/10
195/195 ━━━━━━━━━━━━━━━━━━━━ 186s 956ms/step - accuracy: 0.9889 - loss: 0.0319 - val_accuracy: 0.9980 - val_loss: 0.0057
Epoch 5/10
195/195 ━━━━━━━━━━━━━━━━━━━━ 192s 983ms/step - accuracy: 0.9937 - loss: 0.0188 - val_accuracy: 0.9987 - val_loss: 0.0053
Epoch 6/10
195/195 ━━━━━━━━━━━━━━━━━━━━ 190s 975ms/step - accuracy: 0.9947 - loss: 0.0162 - val_accuracy: 0.9967 - val_loss: 0.0080
Epoch 7/10
195/195 ━━━━━━━━━━━━━━━━━━━━ 189s 970ms/step - accuracy: 0.9958 - loss: 0.0175 - val_accuracy: 0.9967 - val_loss: 0.0075
Epoch 8/10
195/195 ━━━━━━━━━━━━━━━━━━━━ 184s 946ms/step - accuracy: 0.9966 -

In [11]:
# Save the final model
model.save(final)

In [12]:
# Evaluate the final model
loss, accuracy = model.evaluate(validation_generator, steps=validation_steps)
print(f"Test accuracy: {accuracy:.4f}")

48/48 ━━━━━━━━━━━━━━━━━━━━ 28s 580ms/step - accuracy: 0.9989 - loss: 0.0028
Test accuracy: 0.9987


In [13]:
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1 (Conv2D)      │ (None, 112, 112,  │        864 │ input_layer[0][0] │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bn_Conv1            │ (None, 112, 112,  │        128 │ Conv1[0][0]       │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1_relu (ReLU)   │ (None, 112, 112,  │          0 │ bn_Conv1[0][0]    │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │        288 │ Conv1_relu[0][0]  │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │        128 │ expanded_conv_de… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │          0 │ expanded_conv_de… │
│ (ReLU)              │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 112, 112,  │        512 │ expanded_conv_de… │
│ (Conv2D)            │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 112, 112,  │         64 │ expanded_conv_pr… │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand      │ (None, 112, 112,  │      1,536 │ expanded_conv_pr… │
│ (Conv2D)            │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_BN   │ (None, 112, 112,  │        384 │ block_1_expand[0… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_relu │ (None, 112, 112,  │          0 │ block_1_expand_B… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_pad         │ (None, 113, 113,  │          0 │ block_1_expand_r… │
│ (ZeroPadding2D)     │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise   │ (None, 56, 56,    │        864 │ block_1_pad[0][0] │
│ (DepthwiseConv2D)   │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 56, 56,    │        384 │ block_1_depthwis… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 56, 56,    │          0 │ block_1_depthwis… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_project     │ (None, 56, 56,    │      2,304 │ block_1_depthwis

 Total params: 9,285,993 (35.42 MB)

 Trainable params: 2,851,469 (10.88 MB)

 Non-trainable params: 731,584 (2.79 MB)

 Optimizer params: 5,702,940 (21.75 MB)